In [1]:
import numpy as np
import cv2
import time
from scipy.fft import dct, idct

In [2]:
# ICMIC chaotic map
def icmic_map(x, a, iterations, size):
    """Generate a chaotic sequence using ICMIC map."""
    result = np.zeros(size)
    for i in range(size[0]):
        for j in range(size[1]):
            x_n = x
            for _ in range(iterations):
                if x_n != 0:  # Avoid division by zero
                    x_n = np.sin(a / x_n)
                else:
                    x_n = np.random.uniform(-1, 1)  # Handle zero case
                x_n = np.clip(x_n, -1, 1)  # Ensure x_n stays in (-1, 1)
            result[i, j] = x_n
    return result

# Improved Chaos Game Representation (CGR)
def improved_cgr(dna1, dna2, x0, y0, size):
    """Generate chaotic coordinate sequence using improved CGR."""
    vertices = {'A': (0, 0), 'C': (0, 1), 'G': (1, 1), 'T': (1, 0)}
    x, y = x0, y0
    cg_coords = []
    
    for i in range(max(size[0], size[1]) ** 2):  # Generate enough points
        if i < len(dna1) and i < len(dna2):
            e1x, e1y = vertices[dna1[i]]
            e2x, e2y = vertices[dna2[i]]
            x = 0.5 * x + 0.25 * (e1x + e2x)
            y = 0.5 * y + 0.25 * (e1y + e2y)
            cg_coords.append(np.sqrt(x**2 + y**2))  # Distance to origin
        else:
            # Extend sequences if needed
            e1x, e1y = vertices[dna1[i % len(dna1)]]
            e2x, e2y = vertices[dna2[i % len(dna2)]]
            x = 0.5 * x + 0.25 * (e1x + e2x)
            y = 0.5 * y + 0.25 * (e1y + e2y)
            cg_coords.append(np.sqrt(x**2 + y**2))
    
    # Reshape to image size
    cg_coords = np.array(cg_coords)[:size[0] * size[1]].reshape(size)
    return np.clip(cg_coords, 0, 1)  # Normalize to [0, 1]

# Generate encryption parameters from bit planes
def get_encryption_parameters(image):
    """Calculate encryption parameters using 2D-DCT of bit planes."""
    bit_planes = [(image >> i) & 1 for i in range(8)]  # Extract 8 bit planes
    dct_planes = [dct(dct(plane.astype(float), axis=0, norm='ortho'), axis=1, norm='ortho') for plane in bit_planes]
    
    sub1 = np.sum(np.abs(dct_planes[0]) + np.abs(dct_planes[7])) - np.floor(np.sum(np.abs(dct_planes[0]) + np.abs(dct_planes[7])))
    sub2 = np.sum(np.abs(dct_planes[1]) + np.abs(dct_planes[6])) - np.floor(np.sum(np.abs(dct_planes[1]) + np.abs(dct_planes[6])))
    sub3 = np.sum(np.abs(dct_planes[2]) + np.abs(dct_planes[5])) - np.floor(np.sum(np.abs(dct_planes[2]) + np.abs(dct_planes[5])))
    sub4 = np.sum(np.abs(dct_planes[3]) + np.abs(dct_planes[4])) - np.floor(np.sum(np.abs(dct_planes[3]) + np.abs(dct_planes[4])))
    
    return sub1, sub2, sub3, sub4

# DNACNot sequence generation
def generate_dnacnot(image, dna1, dna2, x0, y0, a, subz):
    """Generate DNACNot sequence for encryption."""
    size = image.shape
    # Step 1: Generate CGR sequence
    modna = improved_cgr(dna1, dna2, x0, y0, size)
    
    # Step 2: Generate four chaotic sequences with ICMIC
    sub1, sub2, sub3, sub4 = get_encryption_parameters(image)
    logori = [icmic_map(sub1 + subz, a, 10, size),
              icmic_map(sub2 + subz, a, 10, size),
              icmic_map(sub3 + subz, a, 10, size),
              icmic_map(sub4 + subz, a, 10, size)]
    
    # Step 3: Modify chaotic sequences with MODNA
    dnalog = [np.clip(log + modna, -1, 1) for log in logori]
    
    # Step 4: Convert to integer sequences (simulate quantum binary states)
    dnalogin = [(np.floor((dlog + 1) * 127.5)).astype(np.uint8) for dlog in dnalog]  # Map [-1, 1] to [0, 255]
    
    # Step 5: XOR operations to get final DNACNot sequence
    dnalogin_final = dnalogin[0]
    for i in range(1, 4):
        dnalogin_final = np.bitwise_xor(dnalogin_final, dnalogin[i])
    
    return dnalogin_final

# Encryption function
def encrypt_image(image_path, dna1, dna2, x0=0.5, y0=0.5, a=10, subz=0.1):
    """Encrypt the input image and measure encryption time."""
    # Load image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError("Image not found or invalid format")
    
    start_time = time.time()
    
    # Generate DNACNot sequence
    dnacnot = generate_dnacnot(image, dna1, dna2, x0, y0, a, subz)
    
    # Simulate CNOT operation: XOR the image with DNACNot sequence
    encrypted_image = np.bitwise_xor(image, dnacnot)
    
    encryption_time = time.time() - start_time
    return encrypted_image, encryption_time

# Decryption function
def decrypt_image(encrypted_image, dna1, dna2, x0=0.5, y0=0.5, a=10, subz=0.1):
    """Decrypt the input image and measure decryption time."""
    start_time = time.time()
    
    # Generate the same DNACNot sequence
    dnacnot = generate_dnacnot(encrypted_image, dna1, dna2, x0, y0, a, subz)
    
    # Reverse CNOT operation: XOR with the same DNACNot sequence
    decrypted_image = np.bitwise_xor(encrypted_image, dnacnot)
    
    decryption_time = time.time() - start_time
    return decrypted_image, decryption_time


In [3]:
# Example usage
if __name__ == "__main__":
    # Sample DNA sequences (short for demonstration; in practice, use longer sequences)
    dna1 = "ACGTACGTACGT"
    dna2 = "TGCATGCA"
    
    image_paths = ['baboon.png', 'apple.png', 'jupiter.png', 'cameraman.png', 'medical.png']
    x0, y0 = 0.5, 0.5  # Initial CGR coordinates
    a = 10  # ICMIC parameter
    subz = 0.1  # Modification parameter
    
    for image_path in image_paths:
        try:
            # Encrypt
            encrypted_image, enc_time = encrypt_image(image_path, dna1, dna2, x0, y0, a, subz)
            print(f"Encryption time for {image_path}: {enc_time:.4f} seconds")
            
            # Save encrypted image
            cv2.imwrite(f"encrypted_{image_path}", encrypted_image)
            
            # Decrypt
            decrypted_image, dec_time = decrypt_image(encrypted_image, dna1, dna2, x0, y0, a, subz)
            print(f"Decryption time for {image_path}: {dec_time:.4f} seconds")
            
            # Save decrypted image
            cv2.imwrite(f"decrypted_{image_path}", decrypted_image)
        
        except Exception as e:
            print(f"Error processing {image_path}: {str(e)}")

Encryption time for baboon.png: 90.7904 seconds
Decryption time for baboon.png: 88.1545 seconds
Encryption time for apple.png: 17.7216 seconds
Decryption time for apple.png: 17.1811 seconds
Encryption time for jupiter.png: 30.3356 seconds
Decryption time for jupiter.png: 31.1860 seconds
Encryption time for cameraman.png: 21.4956 seconds
Decryption time for cameraman.png: 21.6534 seconds
Encryption time for medical.png: 16.8480 seconds
Decryption time for medical.png: 16.4151 seconds
